# 自定义层（Layer)和模型（Model)类

使用tensorflow可以通过继承（面对对象编程）layer和model的方法创建自定义的层和模型。<br/>
通过这个方法可以很简单的扩充网络的功能。创建自己的层和模型是必需掌握的。<br/>
例如tensorflow中的Dense和Sequetial（子类）分别继承与Layer和Model（母类）

继承后必须实现的方法是 \__init\__ 和 call

## 自定义层（Layer)

In [ ]:
from tensorflow.keras import layers

class MyDense(layers.Layer):
    
    def __init__(self, input_dim, output_dim):
        # call the super class initialization
        super(MyDense, self).__init__()
        # add_weight (add_variable will be descrepted) => trainable_variables 
        self.kernel = self.add_weight('w', [input_dim, output_dim])
        self.bias = self.add_weight('b', [output_dim])

    def call(self, inputs, training=None):
        return inputs @ self.kernel + self.bias

## 自定义模型（Model)

In [ ]:
import tensorflow as tf

class MyModel(tf.keras.Model):

    def __init__(self):
        super(MyModel, self).__init__()
        # using MyDense customized full connected layer
        self.fc1 = MyDense(28*28, 512)
        self.fc2 = MyDense(512, 128)
        self.fc3 = MyDense(128, 64)
        self.fc4 = MyDense(64, 32)
        self.fc5 = MyDense(32, 10)

    @tf.function
    def call(self, inputs, training=None):
        output = tf.nn.relu(self.fc1(inputs))
        output = tf.nn.relu(self.fc2(output))
        output = tf.nn.relu(self.fc3(output))
        output = tf.nn.relu(self.fc4(output))
        # last layer output is logics
        output =  self.fc5(output)
        return output

## 使用自己的model练丹

依旧使用02中训练和验证数据加载和预处理的方法

In [ ]:
from tensorflow.keras import datasets

#tf.executing_eagerly()

def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255. # cast is ok; but convert_to_tensor is not working-> uint8 can't be converted to float32 tensor
    # due to the self customized model the x input image need to be reshaped from 2x1 [28,28] => 1x1 [28*28]
    x = tf.reshape(x, [28*28])
    y = tf.cast(y, dtype=tf.int64)
    # the output need to one-hot as well
    y = tf.one_hot(y, depth=10)
    return x, y

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path='./mnist.pnz')

train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(preprocess).shuffle(10000).batch(100)

test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test)).map(preprocess).batch(100)

初始化自己的网络

In [ ]:
network = MyModel()

使用compile的方法初始化模型

In [ ]:
from tensorflow.keras import optimizers, losses

network.compile(optimizer=optimizers.Adam(lr=0.01), 
                loss=tf.losses.CategoricalCrossentropy(from_logits=True), 
                metrics=['accuracy'],
                run_eagerly=True)

network.build((None, None, 28*28))
network.summary()

练丹前使用tensorboard去可视化一下训练的过程， 这个和04稍微有些不同，得使用callback的方式

TODO: 模型的graph并没有被创建...奇怪


In [ ]:
import datetime

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = 'logs/' + current_time

#callbacks = [
#    tf.keras.callbacks.TensorBoard(log_dir=log_dir,
#                                   histogram_freq=1,
#                                   write_graph=True, write_grads=False, write_images=True,
#                                   embeddings_freq=0, embeddings_layer_names=None,
#                                   embeddings_metadata=None, embeddings_data=None, update_freq=500)
#]
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='epoch', profile_batch=1)
]


开始练丹了

In [ ]:
network.fit(train_db,
            batch_size=128,
            epochs=10,
            validation_data=test_db, 
            validation_freq=1,
            callbacks = callbacks)

验证，这里应该使用另外的数据集

In [ ]:
network.evaluate(test_db)